In [14]:
# open key

f = open('secret.key')
key = f.read()
f.close()

In [15]:
# load dataframe

import pandas as pd
import requests
import json

df = pd.read_csv('../data/main_dataset_trimmed.csv', dtype={'ZIPCODE':str})

coords = {'progress': 0, 'coords': []}
try:
    f = open('check.txt')
    coords = eval(f.read())
    f.close()
except FileNotFoundError:
    coords = {'progress': 0, 'coords': []}

t = len(df) // 200
c = 0
c_all = coords['progress']
p = coords['progress'] // t

/var/folders/yg/zpnh1p_d4qvg_bz_487cbyp00000gn/T/ipykernel_87822/3495280002.py:7: DtypeWarning: Columns (36,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/main_dataset_trimmed.csv', dtype={'ZIPCODE':str})


In [16]:
def find_coordinates(df_row):
    global t
    global c
    global p
    global c_all
    global coords
    if c == t:
        # checkpoint
        p += 1
        print(f"f{p / 2}% done geocoding")
        coords['progress'] = c_all
        f = open('check.txt', 'w')
        f.write(str(coords))
        f.close()
        c = 0

    addr = (df_row['ST_NUM']+' '+df_row['ST_NAME']+' '+df_row['CITY']+' '+df_row['ZIPCODE']).replace('\n', '')
    addr_nospace = addr.replace(' ', '%20')
    ret = None
    try:
        res = requests.post('https://maps.googleapis.com/maps/api/geocode/json?address=' + addr_nospace + '&key=' + key)
        res = res.json()
    except Exception as e:
        #print("Error", e, "while getting", addr)
        coords['coords'].append(('na', 'na'))
        ret = ('na', 'na')
    if res['status'] != 'OK':
        #print("No location for", addr)
        coords['coords'].append(('na', 'na'))
        ret = ('na', 'na')
    else:
        #print("Found location for", addr)
        lat = res['results'][0]['geometry']['location']['lat']
        long = res['results'][0]['geometry']['location']['lng']
        coords['coords'].append((lat, long))
        ret = (lat, long)
    c += 1
    c_all += 1
     
df.astype(str).iloc[coords['progress']:].apply(find_coordinates, axis=1)

f0.5% done geocoding
f1.0% done geocoding
f1.5% done geocoding
f2.0% done geocoding
f2.5% done geocoding
f3.0% done geocoding
f3.5% done geocoding
f4.0% done geocoding
f4.5% done geocoding
f5.0% done geocoding
f5.5% done geocoding
f6.0% done geocoding
f6.5% done geocoding
f7.0% done geocoding
f7.5% done geocoding
f8.0% done geocoding
f8.5% done geocoding
f9.0% done geocoding
f9.5% done geocoding
f10.0% done geocoding
f10.5% done geocoding
f11.0% done geocoding
f11.5% done geocoding
f12.0% done geocoding
f12.5% done geocoding
f13.0% done geocoding
f13.5% done geocoding
f14.0% done geocoding
f14.5% done geocoding
f15.0% done geocoding
f15.5% done geocoding
f16.0% done geocoding
f16.5% done geocoding
f17.0% done geocoding
f17.5% done geocoding
f18.0% done geocoding
f18.5% done geocoding
f19.0% done geocoding
f19.5% done geocoding
f20.0% done geocoding
f20.5% done geocoding
f21.0% done geocoding
f21.5% done geocoding
f22.0% done geocoding
f22.5% done geocoding
f23.0% done geocoding
f23.5% 

0        None
1        None
2        None
3        None
4        None
         ... 
66288    None
66289    None
66290    None
66291    None
66292    None
Length: 66293, dtype: object

In [28]:
# clean up and save

#g = open('check2.txt', 'w')
#g.write(str(coords))
#g.close()

import numpy as np

f = open('check2.txt', 'r')
coords = eval(f.read())
f.close()

print(coords['coords'][0])
type(coords['coords'][0])
type(coords['coords'][0][0])

# convert NA
for i,item in enumerate(coords['coords']):
    if item[0] == 'na':
        coords['coords'][i] = "(pd.NA, pd.NA)"
    else:
        coords['coords'][i] = str(item)

print(len(coords['coords']))

df['location'] = np.array(coords['coords'])

print(df['location'].head)

df.to_csv('../data/main_dataset_trimmed_geocoded.csv')

(42.379896, -71.03105599999999)
66293
<bound method NDFrame.head of 0         (42.379896, -71.03105599999999)
1         (42.379896, -71.03105599999999)
2         (42.379896, -71.03105599999999)
3               (42.3801075, -71.0307385)
4               (42.3801075, -71.0307385)
                       ...               
66288           (42.3463185, -71.1662042)
66289           (42.3462372, -71.1631076)
66290           (42.3462372, -71.1631076)
66291    (42.34079570000001, -71.1663499)
66292    (42.3407922, -71.16636989999999)
Name: location, Length: 66293, dtype: object>
